<a href="https://colab.research.google.com/github/rajeshpahari/Deep-Learning/blob/main/Pytorch_Cross_Validations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import Libraries
import numpy as np
import torch
import torch.nn as nn
import seaborn as sns

In [2]:
iris=sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
#Convert dataframe as tensor
data=torch.tensor(iris[iris.columns[0:4]].values).float()
#data
#Labels conversion
labels=torch.zeros(len(data),dtype=torch.long)
labels[iris.species=='versicolor']=1
labels[iris.species=='virginica']=2

In [26]:
#iris.species.unique()

In [15]:
##########Seperate Data into train and test set
#No dev set only train and test set

#How many training examples
propTraining=.8 #proportion of training data
nTraining=int(propTraining*len(labels))

# TraintestBool=np.zeros(len(labels),dtype=bool)
trainSet=np.random.choice(range(len(labels)),nTraining,replace=False)
TraintestBool[trainSet]=True
TraintestBool
print(torch.mean(labels.float()))
print('\n')
print(torch.mean(labels[TraintestBool].float()))
print('\n')
print(torch.mean(labels[~TraintestBool].float()))


tensor(1.)


tensor(0.9930)


tensor(1.1429)


In [16]:
############Create Model

ANNiris=nn.Sequential(
    nn.Linear(4,64),
    nn.ReLU(),
    nn.Linear(64,64),
    nn.ReLU(),
    nn.Linear(64,3)
)

#Lossfunction
lossfunc=nn.CrossEntropyLoss()

#optimizer
optimizer=torch.optim.SGD(ANNiris.parameters(),lr=.01)



In [17]:
print(data.shape)
print('/n')
print(data[TraintestBool,:].shape)
print('/n')
print(data[~TraintestBool,:].shape)

torch.Size([150, 4])
/n
torch.Size([143, 4])
/n
torch.Size([7, 4])


In [26]:
######################Train and test the Model
numepochs=1000

#Initialize losses and accuracy
losses=np.zeros(numepochs)
ongoingACC=[]

#Loop
for epochi in range(numepochs):

  #forward pass
  yHat=ANNiris(data[TraintestBool,:])

  #Loss computation
  loss=lossfunc(yHat,labels[TraintestBool])
  losses[epochi]=loss

  ##Accuracy
  ongoingACC.append(100*torch.mean((torch.argmax(yHat,axis=1)==labels[TraintestBool]).float()))
  



  #Backprop
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()


In [29]:
########Compute Train and test accuracy
#prediction with training data
pred=ANNiris(data[TraintestBool,:])
trainacc=(100*torch.mean((torch.argmax(pred,axis=1)==labels[TraintestBool]).float()))
print('Train Accuracy: %g%% ' %trainacc)

#Test Accuracy
predTest=ANNiris(data[~TraintestBool,:])
testacc=(100*torch.mean((torch.argmax(predTest,axis=1)==labels[~TraintestBool]).float()))
print('Test Accuracy: %g%%' %testacc)

Train Accuracy: 97.9021% 
Test Accuracy: 100%
